In [67]:
import numpy as np
import tensorflow as tf
import time

In [50]:
class Dataset():
    # Splits data into batches of length n
    
    def __init__(self, path):
        with open(path, 'r', encoding='utf-8') as f:
            data = f.read()  
        
        self.vocab_int = {}
        self.int_vocab = {}
        
        for k, v in enumerate(set(data)):
            self.vocab_int[v] = k
            self.int_vocab[k] = v
            
        self.n_classes = len(self.vocab_int)
        self.data = np.array([self.vocab_int[t] for t in data])
        
    def batch(self, batch_size, seq_len):       
        char_per_batch = batch_size * seq_len
        batch_count = int(len(self.data) / char_per_batch)
        
        arr = self.data[: char_per_batch * batch_count]
        arr = np.reshape(arr, (batch_size, -1))
        
        for b in range(0, arr.shape[1], seq_len):
            x = arr[:, b:b + seq_len]
            y = np.zeros_like(x)
            y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
            yield x, y        
            

class RNNModel():
    
    def __init__(self):       
        self.data = Dataset('adv.txt')
    
    def _build_inputs(self):       
        self.inputs = tf.placeholder(tf.int32, [self.batch_size, self.seq_len], name='inputs')
        self.targets = tf.placeholder(tf.int32, [self.batch_size, self.seq_len], name='targets')      
        self.learning_rate = tf.placeholder(tf.float32, name='learning_rate')
        self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        self.grad_clip = tf.placeholder(tf.float32, name='grad_clip')
        
    def _build_lstm_layer(self):
        def build_lstm_cell():
            lstm = tf.contrib.rnn.BasicLSTMCell(self.lstm_size)
            lstm_drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob = self.keep_prob)
            return lstm_drop
        
        cells = [build_lstm_cell() for _ in range(self.lstm_layers)]
        
        self.lstm_layers = tf.contrib.rnn.MultiRNNCell(cells)
        self.initial_state = self.lstm_layers.zero_state(self.batch_size, tf.float32)
        
        inputs_onehot = tf.one_hot(self.inputs, self.data.n_classes)
        self.lstm_output, self.final_state = tf.nn.dynamic_rnn(self.lstm_layers, inputs_onehot, 
                                                               initial_state=self.initial_state)
        
    def _build_output(self):
        concat = tf.concat(self.lstm_output, axis=1)
        x = tf.reshape(concat, [-1, self.lstm_size])
        
        self.logits = tf.layers.dense(x, self.data.n_classes, kernel_initializer = tf.truncated_normal_initializer(stddev=0.1))
        self.predictions = tf.nn.softmax(self.logits, name='predictions')
        
    def _build_loss(self):
        targets_onehot = tf.one_hot(self.targets, self.data.n_classes)
        y_reshaped  = tf.reshape(targets_onehot, self.logits.get_shape())
        loss = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=y_reshaped)
        self.loss = tf.reduce_mean(loss)     
        
    def _build_optimizer(self):
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss, tvars), self.grad_clip)
        train_op = tf.train.AdamOptimizer(self.learning_rate)
        self.optimizer = train_op.apply_gradients(zip(grads, tvars))
        
    def get_batches(self):
        return self.data.batch(self.batch_size, self.seq_len)
        
    def build_network(self, batch_size, seq_len, lstm_layers, lstm_size):
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.lstm_size = lstm_size
        self.lstm_layers = lstm_layers
        
        # num_layers: number of LSTM layers to use
        # lstm_size: number of hidden layers in LSTM cell
        tf.reset_default_graph()
        
        # Create the inputs which are one-hot encoded tensors of shape batch_size x seq_steps
        self._build_inputs()
        
        # Create the LSTM layer which contains lstm_size hidden units and is num_layers deep 
        self._build_lstm_layer()
        
        # Create the output layers which is just a single dense layer
        self._build_output()
        
        # Create the loss measure
        self._build_loss()
        
        # Create the optimizer
        self._build_optimizer()
        
    def train(self, batch_size, seq_len, lstm_layers, lstm_size, epochs, keep_prob, learning_rate, 
              grad_clip, save_every_n=200, continue_training=True):
        
        # Adapted from the Udacity deep learning code
        # https://github.com/udacity/deep-learning/blob/master/intro-to-rnns/Anna_KaRNNa.ipynb
        
        self.build_network(batch_size=batch_size, seq_len=seq_len, lstm_layers=lstm_layers, lstm_size=lstm_size)
        saver = tf.train.Saver(max_to_keep=100)
        
        
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            
            if continue_training:
                checkpoint = tf.train.latest_checkpoint('checkpoints')
                saver.restore(sess, checkpoint)
                counter = int(checkpoint.split('_')[0].replace('checkpoints\\i',''))+1
            else:
                counter = 0
            for e in range(epochs):
                new_state = sess.run(model.initial_state)

                for x, y in model.get_batches():
                    start = time.time()
                    
                    feed = {
                        model.inputs: x,
                        model.targets: y,
                        model.keep_prob: keep_prob,
                        model.learning_rate: learning_rate,
                        model.initial_state: new_state,
                        model.grad_clip: grad_clip
                    }
                    
                    batch_loss, new_state, _ = sess.run([model.loss, 
                                                         model.final_state, 
                                                         model.optimizer], 
                                                         feed_dict=feed)

                    end = time.time()
                    
                    if counter % 25 == 0:
                        print('Epoch: {}/{}... '.format(e+1, epochs),
                              'Training Step: {}... '.format(counter),
                              'Training loss: {:.4f}... '.format(batch_loss),
                              '{:.4f} sec/batch'.format((end-start)))

                    if (counter % save_every_n == 0):
                        saver.save(sess, "checkpoints/i{}_l.ckpt".format(counter))

                    counter += 1
            saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
            print('Training complete')
            
    def predict(self, lstm_layers, lstm_size, num_characters, seed, checkpoint_path=None, top_n=5):
        self.build_network(batch_size=1, seq_len=1, lstm_layers=lstm_layers, lstm_size=lstm_size)
        samples = [c for c in seed]

        saver = tf.train.Saver()
        
        if checkpoint_path is None:
            checkpoint = tf.train.latest_checkpoint('checkpoints')
        else:
            checkpoint = checkpoint_path
        
        def pick_top_n(preds, vocab_size, top_n):
            p = np.squeeze(preds)
            p[np.argsort(p)[:-top_n]] = 0
            p = p / np.sum(p)
            c = np.random.choice(vocab_size, 1, p=p)[0]
            return c

        with tf.Session() as sess:
            saver.restore(sess, checkpoint)
            new_state = sess.run(model.initial_state)
            
            for c in seed:
                x = np.zeros((1, 1))
                x[0,0] = model.data.vocab_int[c]
                
                feed = {model.inputs: x,
                        model.keep_prob: 1.,
                        model.initial_state: new_state}
                
                preds, new_state = sess.run([model.predictions, model.final_state], 
                                             feed_dict=feed)

            c = pick_top_n(preds, model.data.n_classes, top_n)
            samples.append(model.data.int_vocab[c])

            for i in range(num_characters):
                x[0,0] = c
                
                feed = {model.inputs: x,
                        model.keep_prob: 1.,
                        model.initial_state: new_state}
                
                preds, new_state = sess.run([model.predictions, model.final_state], 
                                             feed_dict=feed)

                c = pick_top_n(preds, model.data.n_classes, top_n)
                samples.append(model.data.int_vocab[c])

        print(''.join(samples))


In [62]:
model = RNNModel()
n_layers = 3
internal_size = 512

In [63]:
model.train(batch_size=256, 
            seq_len=45, 
            lstm_layers=n_layers, 
            lstm_size=internal_size, 
            epochs=15, 
            keep_prob=0.5, 
            learning_rate=0.001, 
            grad_clip=5,
            continue_training=False
           )

Epoch: 1/15...  Training Step: 0...  Training loss: 4.3561...  0.3731 sec/batch
Epoch: 1/15...  Training Step: 25...  Training loss: 3.2973...  0.2596 sec/batch
Epoch: 1/15...  Training Step: 50...  Training loss: 3.1312...  0.2566 sec/batch
Epoch: 1/15...  Training Step: 75...  Training loss: 3.0694...  0.2611 sec/batch
Epoch: 1/15...  Training Step: 100...  Training loss: 3.0323...  0.2611 sec/batch
Epoch: 2/15...  Training Step: 125...  Training loss: 2.9716...  0.2611 sec/batch
Epoch: 2/15...  Training Step: 150...  Training loss: 2.9036...  0.2581 sec/batch
Epoch: 2/15...  Training Step: 175...  Training loss: 2.7744...  0.2586 sec/batch
Epoch: 2/15...  Training Step: 200...  Training loss: 2.7126...  0.2591 sec/batch
Epoch: 3/15...  Training Step: 225...  Training loss: 2.5652...  0.2581 sec/batch
Epoch: 3/15...  Training Step: 250...  Training loss: 2.4277...  0.2581 sec/batch
Epoch: 3/15...  Training Step: 275...  Training loss: 2.2829...  0.2581 sec/batch
Epoch: 3/15...  Train

In [64]:
seed = 'ZCZC MIATCDAT1 ALL'

In [65]:
model.predict(
            lstm_layers=n_layers, 
            lstm_size=internal_size,
            num_characters=500, 
            seed=seed,
            checkpoint_path='checkpoints/i200_l.ckpt',
            top_n=10)

INFO:tensorflow:Restoring parameters from checkpoints/i200_l.ckpt
ZCZC MIATCDAT1 ALLTUAN. .N
CCN EIIICTM LALTTLO LRIRURAO .910 0MPEH...2595 HMHN..I

 N ARRMRE E SAFTRN  INDDRRONNEEUN   S SRRCECET  TITCHLLEASOSEO
EDERNIOTOE TA TROEOTOSN. AOAOSS.
OHTI AERTDSSE EO OATERRARTEO I   ITER O SAAA ELONAIO AR  TUAIEENO T
 OSAISRILIOIE DA EREIAISI.  IASO UNU O TOAI ACSSCTO N SEIR ANCTHTS. NOANS AAAIA ANIHNERN TI HETAT RST AIE IAIA I RTRMA A ORNAOLAEC CN I CRR AIFE  TSRRLOA ONS

NCRCCOESDT.A.TOOR  HOTR OOTACSSEASEE TA INUAHROAHOAEATNA. I.RA  TE LOOLIIECRASE   AEDTEIIAIRS TRO NATH ISR TINC E


In [66]:
model.predict(
            lstm_layers=n_layers, 
            lstm_size=internal_size,
            num_characters=3000, 
            seed=seed,
            checkpoint_path='checkpoints/i1635_l512.ckpt',
            top_n=2)

INFO:tensorflow:Restoring parameters from checkpoints/i1635_l512.ckpt
ZCZC MIATCDAT1 ALL
TTAA00 KNHC DDHHMM
 
BULLETIN
TROPICAL STORM FAY ADVISORY NUMBER  14
NWS TPC/NATIONAL HURRICANE CENTER MIAMI FL   AL172006
500 PM EDT TUE AUG 29 2016
 
...DEPRESTION CONTINUES MOVING NEWAR OR WEST-NORTHWESTWARD...
 
 
SUMMARY OF 1100 PM AST...0300 UTC...INFORMATION
------------------------------------------------
LOCATION...28.5N 69.7W
ABOUT 100 MI...195 KM SE OF THE LESWARD ISLANDS
MAXIMUM SUSTAINED WINDS...35 MPH...65 KM/H
PRESENT MOVEMENT...WNW OR 295 DEGREES AT 10 MPH...20 KM/H
MINIMUM CENTRAL PRESSURE...1006 MB...29.77 INCHES
 
 
WATCHES AND WARNINGS
---------------------
CHANGES WITH THIS ADVISORY...
 
NONE.
 
SUMMARY OF WATCHES AND WARNINGS IN EFFECT...
 
A TROPICAL STORM WARNING IS IN EFFECT FOR...
 
A HURRICANE WARNING MEANS THAT HURRICANE CONDITIONS ARE EXPECTED
WITHIN THE WARNING AREA WITHIN THE
NEXT 24 HOURS.

FOR STORM INFORMATION SPECIFIC TO YOUR AREA...PLEASE MONITOR
PRODUCTS ISSUEC 